In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import sys
sys.path.append('..')
from security_game.target import Target
from security_game.green_security_game import GreenSecurityGame
from security_game.infra_security_game import InfraSecurityGame

from solvers.mip import mip
from solvers.nash import nash
from solvers.double_oracle import double_oracle
from solvers.double_oracle_sf import double_oracle_sf
from solvers.no_regret import regret_matching
from solvers.simple_sse_lp import solve_sse_lp
from solvers.nfg_sse_lp import solve_general_sum_normal_form

from utils.random_utils import generate_random_utility_matrix_like, generate_random_target_utility_matrix_like, generate_random_schedule_mapping_like

import time
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
import copy

In [2]:
seeds = [1,2,3,4,5,6,7,8,9,10]

# GSG

In [3]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

## Simple SSE LP - Random target values only

In [4]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": True,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 0, 
}

# general_sum_kwargs = {
#     "general_sum": True,
#     "attacker_animal_value":  1, 
#     "defender_animal_value": 1, 
#     "defender_step_cost": 0, 
# }



supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(gsg.schedule_form_dict)
    sdict_copy["target_utilities"] = generate_random_target_utility_matrix_like(gsg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    start = time.time()
    _, coverage, du = solve_sse_lp(sdict_copy)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2455389
Academic license 2455389 - for non-commercial use only - registered to jc___@columbia.edu
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [5]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.324292,3,0.025184
1,2,8,2,10,7,1,-0.247220,4,0.013263
2,3,8,2,10,7,1,-0.323111,3,0.013160
3,4,8,2,10,7,1,-0.473990,0,0.012574
4,5,8,2,10,7,1,-0.121722,1,0.013433
5,6,8,2,10,7,1,-0.300946,2,0.013194
6,7,8,2,10,7,1,-0.102518,3,0.014259
7,8,8,2,10,7,1,-0.059489,2,0.012332
8,9,8,2,10,7,1,-0.133474,1,0.008522
9,10,8,2,10,7,1,-0.255545,2,0.007251


In [10]:
[round(x,4) for x in df.support]

[3, 4, 3, 0, 1, 2, 3, 2, 1, 2]

In [10]:
df.to_csv("GSG_SIMPLE_SSE_RANDOM_TARGET_VALUES.csv")

## Simple SSE LP - Entirely Random schedules

## General SSE LP - Random Matrix

In [17]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

In [18]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 1.17, 
}


supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(gsg.schedule_form_dict)
    sdict_copy["defender_utility_matrix"] = generate_random_utility_matrix_like(gsg.schedule_form_dict["defender_utility_matrix"])
    sdict_copy["attacker_utility_matrix"] = generate_random_utility_matrix_like(gsg.schedule_form_dict["attacker_utility_matrix"])
    print(sdict_copy["defender_utility_matrix"])
    print(sdict_copy["attacker_utility_matrix"])
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(sdict_copy["defender_utility_matrix"],sdict_copy["attacker_utility_matrix"])
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
[[-0.59742449 -0.30462902 -0.99988959 ... -0.6664102  -0.61697736
  -0.47984898]
 [-0.59532724 -0.33851791 -0.80263039 ... -0.46066433 -0.86447635
  -0.80876115]
 [-0.22699486 -0.06528099 -0.69743348 ... -0.96229815 -0.83605285
  -0.15227815]
 ...
 [-0.18044965 -0.23259856 -0.27388863 ... -0.77199391 -0.91501212
  -0.32172104]
 [-0.67236926 -0.97400265 -0.63845984 ... -0.22998735 -0.75451045
  -0.71411983]
 [-0.95733837 -0.56400246 -0.112748   ... -0.0680325  -0.21442391
  -0.89149185]]
[[0.10824188 0.04927971 0.01750998 ... 0.04522027 0.05124552 0.09363286]
 [0.02349564 0.01052974 0.09500784 ... 0.0500015  0.03642908 0.03477958]
 [0.03778414 0.02073653 0.02475779 ... 0.05001562 0.06074748 0.00615572]
 ...
 [0.11012689 0.05076613 0.05476934 ... 0.07319687 0.03734724 0.0317944 ]
 [0.03826706 0.01332646 0.04585729 ... 0.10538474 0.00466471 0.07733857]
 [0.05479617 0.06355151 0.04298812 ... 0.01119069 0.1166606  0.02232429]]
starting seed: 2
[[-0.57910885 -0.97497191 -0.4

In [19]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.034781,2,0.519425
1,2,8,2,10,7,1,-0.035010,3,0.436632
2,3,8,2,10,7,1,-0.034768,2,0.601936
3,4,8,2,10,7,1,-0.034802,2,0.567851
4,5,8,2,10,7,1,-0.034800,2,0.451001
5,6,8,2,10,7,1,-0.034754,2,0.502337
6,7,8,2,10,7,1,-0.034720,1,0.499972
7,8,8,2,10,7,1,-0.034861,2,0.481408
8,9,8,2,10,7,1,-0.034830,2,0.619688
9,10,8,2,10,7,1,-0.034765,2,0.478741


In [22]:
[round(x,4) for x in df.runtime]

[0.5194, 0.4366, 0.6019, 0.5679, 0.451, 0.5023, 0.5, 0.4814, 0.6197, 0.4787]

In [17]:
df.to_csv("GSG_NFG_SSE_RANDOM_MATRIX.csv")

In [19]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 1.17, 
}


supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)

for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    
    # sdict_copy = copy.deepcopy(gsg.schedule_form_dict)
    # sdict_copy["defender_utility_matrix"] = generate_random_utility_matrix_like(gsg.schedule_form_dict["defender_utility_matrix"])
    # sdict_copy["attacker_utility_matrix"] = generate_random_utility_matrix_like(gsg.schedule_form_dict["attacker_utility_matrix"])
    defender_actions,defender_costs = generate_defender_actions_with_costs(gsg.schedule_form_dict["schedules"])
    attacker_actions = [0,1,2,3,4,5,6,7,8,9]
    random_target_utilities = generate_random_target_utility_matrix_like(gsg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    attacker_matrix, defender_matrix = generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, random_target_utilities, general_sum=True)
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(defender_matrix,attacker_matrix)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [21]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.324255,4,0.457268
1,2,8,2,10,7,1,-0.248860,4,0.579033
2,3,8,2,10,7,1,-0.323056,3,0.420156
3,4,8,2,10,7,1,-0.472870,1,0.419234
4,5,8,2,10,7,1,-0.122495,2,0.374988
5,6,8,2,10,7,1,-0.300856,2,0.297631
6,7,8,2,10,7,1,-0.103833,3,0.438689
7,8,8,2,10,7,1,-0.061733,2,0.366887
8,9,8,2,10,7,1,-0.134686,1,0.406830
9,10,8,2,10,7,1,-0.115241,4,0.412560


In [22]:
df.to_csv("GSG_NFG_SSE_RANDOM_TARGET_VALUES.csv")

In [7]:
    def generate_defender_actions_with_costs(schedule_dict):
        """
        Generates all possible joint defender actions from a dictionary mapping
        each defender to their list of (schedule, cost) tuples.

        Defenders with no schedules are assigned an empty set with 0 cost as a placeholder.

        Returns:
            defender_actions: list of lists of sets (each inner list is one full defender action)
            defender_costs: list of floats representing the total cost for each joint action
        """
        sorted_defenders = sorted(schedule_dict.keys())
        # print(schedule_dict)
        # Replace empty schedule lists with a dummy no-op schedule
        schedule_lists = [
            schedule_dict[d] if schedule_dict[d] else [({}, 0)]
            for d in sorted_defenders
        ]
        # print(schedule_lists)
        all_combinations = list(itertools.product(*schedule_lists))

        defender_actions = []
        defender_costs = []

        for combo in all_combinations:
            schedules = [item[0] for item in combo]
            # print(combo)
            total_cost = sum(item[1] for item in combo)
            defender_actions.append(schedules)
            defender_costs.append(total_cost)

        return defender_actions, defender_costs

    def generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, target_utility_matrix, general_sum):
        """
        Builds utility matrices for all combinations of defender and attacker actions,
        then normalizes both matrices together based on the largest absolute value.

        Parameters:
            attacker_actions (list[int]): List of target nodes (attacker action).
            defender_actions (list[list[set]]): Each outer list is one defender action;
                inner list is per-defender selected schedule (a set of targets).
            defender_costs (list[float]): Cost associated with each defender action.
            target_utility_matrix (np.ndarray): 4 x num_targets array:
                [0]: defender utility if uncovered
                [1]: defender utility if covered
                [2]: attacker utility if covered
                [3]: attacker utility if uncovered

        Returns:
            tuple:
                - attacker_util_matrix (np.ndarray): shape (len(defender_actions), len(attacker_actions))
                - defender_util_matrix (np.ndarray): same shape
        """
        num_defender_actions = len(defender_actions)
        num_attacker_actions = len(attacker_actions)

        defender_util_matrix = np.zeros((num_defender_actions, num_attacker_actions))
        attacker_util_matrix = np.zeros((num_defender_actions, num_attacker_actions))

        for i, d_action in enumerate(defender_actions):
            target_coverage_count = {}
            for schedule in d_action:
                for t in schedule:
                    target_coverage_count[t] = target_coverage_count.get(t, 0) + 1

            for j, atk_target in enumerate(attacker_actions):
                num_covers = target_coverage_count.get(atk_target, 0)

                if num_covers == 0:
                    defender_util = target_utility_matrix[0][j]  # uncovered
                    attacker_util = target_utility_matrix[3][j]  # uncovered
                else:
                    defender_util = target_utility_matrix[1][j]  # covered
                    attacker_util = target_utility_matrix[2][j]  # covered

                defender_util_matrix[i, j] = defender_util - defender_costs[i]
                # defender_util_matrix[i, j] = defender_util
                attacker_util_matrix[i, j] = attacker_util

        # if general_sum:
            # Normalize attacker and defender matrices together
        max_abs_value = max(
            np.abs(defender_util_matrix).max(),
            np.abs(attacker_util_matrix).max()
        )

        if max_abs_value > 0:
            defender_util_matrix = defender_util_matrix / max_abs_value
            attacker_util_matrix = attacker_util_matrix / max_abs_value

        return attacker_util_matrix, defender_util_matrix

In [29]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 1.17, 
}


supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             random_target_values=False,
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)

for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)

    random_schedules = generate_random_schedule_mapping_like(gsg.schedule_form_dict["schedules"],48)
    defender_actions, defender_costs = generate_defender_actions_with_costs(random_schedules)
    attacker_actions = [0,1,2,3,4,5,6,7,8,9]
    random_target_utilities = generate_random_target_utility_matrix_like(gsg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    attacker_matrix, defender_matrix = generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, random_target_utilities, general_sum=True)
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(defender_matrix,attacker_matrix)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)


starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [9]:
gsg.schedule_form_dict["target_utilities"]

array([[-16503.20603015,  -5193.31658291,  -8424.71356784,
        -12002.33165829,  -5597.24120603,  -6058.86934673,
        -13560.32663317, -13906.54773869,  -2769.76884422,
         -7847.67839196],
       [ -3300.64120603,  -1038.66331658,  -1684.94271357,
         -2400.46633166,  -1119.44824121,  -1211.77386935,
         -2712.06532663,  -2781.30954774,   -553.95376884,
         -1569.53567839],
       [   337.73869347,    212.56281407,    302.18301373,
           258.64967764,    156.23836827,    213.19269702,
           393.81980819,    379.47271713,     69.42657761,
           267.67663265],
       [  1688.69346734,   1062.81407035,   1510.91506863,
          1293.24838819,    781.19184134,   1065.9634851 ,
          1969.09904096,   1897.36358564,    347.13288803,
          1338.38316325]])

In [30]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.364794,2,0.433395
1,2,8,2,10,7,1,-0.041760,1,0.437731
2,3,8,2,10,7,1,-0.335759,3,0.362280
3,4,8,2,10,7,1,-0.137561,2,0.341587
4,5,8,2,10,7,1,-0.262375,2,0.380102
5,6,8,2,10,7,1,-0.069592,3,0.350575
6,7,8,2,10,7,1,-0.077671,1,0.364267
7,8,8,2,10,7,1,-0.295773,3,0.501500
8,9,8,2,10,7,1,-0.237512,2,0.391941
9,10,8,2,10,7,1,-0.133573,4,0.385948


In [31]:
df.to_csv("GSG_NFG_SSE_RANDOM_SCHEDULES_AND_TARGET_VALUES.csv")

# ISG

In [11]:
gdf = gpd.read_file("chinatown_infra.geojson")

# Step 1: Handle relevant columns
infra_columns = [
    "id", "name", "power", "man_made", "amenity",
    "generator:method", "generator:source", "geometry"
]
available_columns = [col for col in infra_columns if col in gdf.columns]
gdf = gdf[available_columns].copy()

# Step 2: Extract generator type if present
gdf["generator_type"] = gdf.get("generator:method")
if "generator_type" not in gdf.columns or gdf["generator_type"].isnull().all():
    gdf["generator_type"] = gdf.get("generator:source")

# Step 3: Construct unified 'type' column
gdf["type"] = gdf.get("power")
if "amenity" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["amenity"])
if "man_made" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["man_made"])

# Step 4: Refine generator classification (solar vs. other)
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "photovoltaic"), "type"] = "solar_generator"
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "solar"), "type"] = "solar_generator"

# Step 5: Drop raw columns now that 'type' is finalized
df_simple = gdf[["id", "name", "type", "geometry"]].copy()

# Step 6: Separate nodes and ways
df_nodes = df_simple[df_simple["id"].str.contains("node")].copy()
df_nodes["x"] = df_nodes.geometry.x
df_nodes["y"] = df_nodes.geometry.y
df_nodes = df_nodes.drop(columns=["geometry"])

df_ways = df_simple[df_simple["id"].str.contains("way")].copy()
df_ways = df_ways.set_geometry("geometry").to_crs("EPSG:32618")
df_ways["centroid"] = df_ways.geometry.centroid
df_ways = df_ways.set_geometry("centroid").to_crs("EPSG:4326")
df_ways["x"] = df_ways.geometry.x
df_ways["y"] = df_ways.geometry.y
df_ways = df_ways.drop(columns=["geometry", "centroid"])

# Step 7: Combine nodes and ways
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
ny_blocks_gdf =  gpd.read_file("tl_2020_36_tabblock20.shp")
INFRA_WEIGHTS = {
    # Power Infrastructure
    "plant": 1.5,
    "generator": 1.35,
    "solar_generator": 0.95,
    "substation": 1.45,
    "transformer": 1.25,
    "tower": 1.1,
    "pole": 0.85,
    "line": 1.0,
    "minor_line": 0.9,
    "cable": 0.95,
    "switchgear": 1.2,
    "busbar": 0.8,
    "bay": 0.85,
    "converter": 1.05,
    "insulator": 0.75,
    "portal": 0.75,
    "connection": 0.7,
    "compensator": 1.0,
    "rectifier": 0.95,
    "inverter": 0.95,
    "storage": 0.9,

    # Healthcare
    "hospital": 1.5,
    "clinic": 1.35,

    # Education
    "school": 1.25,
    "university": 1.4,

    # Water & Sanitation
    "water_works": 1.45,
    "wastewater_plant": 1.4,

    # Government & Emergency Services
    "fire_station": 1.3,
    "police": 1.4,
    "courthouse": 1.2,

    # Critical Infrastructure
    "bunker_silo": 1.0,

    # Communications
    "communications_tower": 1.25,
}


## ISG Simple SSE LP

In [12]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": True,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_feature_value":  1, 
    "defender_feature_value": 100, 
    "defender_step_cost": 0,
    "alpha":.5
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

num_timesteps_li = []
num_defenders_li = []
supports=[]
dts = []

dus = []
runtimes = []
i=0

isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
isg.generate(num_attackers=1, 
             num_defenders=3, 
             home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             generate_actions=False, 
             force_return=True, 
             defense_time_threshold=1, 
             **schedule_form_kwargs,
             **general_sum_kwargs, 
            )
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(isg.schedule_form_dict)
    sdict_copy["target_utilities"] = generate_random_target_utility_matrix_like(isg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    
    start = time.time()
    _, coverage, du = solve_sse_lp(sdict_copy)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [13]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [3 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,defense_time_threshold,defender_utility,support,runtime
0,1,8,3,1,-0.560781,2,0.026646
1,2,8,3,1,-0.227405,2,0.067207
2,3,8,3,1,-0.159770,1,0.061839
3,4,8,3,1,-0.046301,1,0.036750
4,5,8,3,1,-0.235854,1,0.041341
5,6,8,3,1,-0.063357,4,0.037029
6,7,8,3,1,-0.111064,2,0.052423
7,8,8,3,1,-0.080143,1,0.052389
8,9,8,3,1,-0.068013,5,0.055207
9,10,8,3,1,-0.109245,1,0.039587


In [16]:
[round(x,4) for x in df.support]

[2, 2, 1, 1, 1, 4, 2, 1, 5, 1]

In [9]:
df.to_csv("ISG_SIMPLE_SSE_RANDOM_TARGET_VALUES.csv")

In [33]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_feature_value":  1, 
    "defender_feature_value": 100, 
    "defender_step_cost": 2,
    "alpha":.5
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

supports=[]
dts = []

dus = []
runtimes = []

isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
isg.generate(num_attackers=1, 
             num_defenders=3, 
             home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             generate_actions=False, 
             force_return=True, 
             defense_time_threshold=2, 
             **schedule_form_kwargs,
             **general_sum_kwargs, 
            )
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(isg.schedule_form_dict)
    sdict_copy["defender_utility_matrix"] = generate_random_utility_matrix_like(isg.schedule_form_dict["defender_utility_matrix"])
    sdict_copy["attacker_utility_matrix"] = generate_random_utility_matrix_like(isg.schedule_form_dict["attacker_utility_matrix"])
    print(sdict_copy["defender_utility_matrix"])
    print(sdict_copy["attacker_utility_matrix"])
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(sdict_copy["defender_utility_matrix"],sdict_copy["attacker_utility_matrix"])
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
[[-0.59169758 -0.2947369  -0.99988802 ... -0.21599834 -0.05198398
  -0.69312926]
 [-0.32215328 -0.14193542 -0.12409883 ... -0.22722385 -0.89893236
  -0.56147156]
 [-0.11040248 -0.71252508 -0.71824181 ... -0.35008476 -0.4958768
  -0.07515594]
 ...
 [-0.62150302 -0.07923594 -0.22096152 ... -0.34961872 -0.33088097
  -0.92943706]
 [-0.81947367 -0.43827116 -0.51085942 ... -0.81773173 -0.65734987
  -0.53180536]
 [-0.09006962 -0.92660242 -0.53653057 ... -0.14301337 -0.96011702
  -0.42188867]]
[[0.00270674 0.00285909 0.00902488 ... 0.00370563 0.00952263 0.00672378]
 [0.00855286 0.0079319  0.00828676 ... 0.00071188 0.00067955 0.0012529 ]
 [0.00306844 0.00713365 0.00960149 ... 0.00160479 0.00218471 0.00463471]
 ...
 [0.00350636 0.00487196 0.00447379 ... 0.00628288 0.00880423 0.00918377]
 [0.00524711 0.00264302 0.00951103 ... 0.00617418 0.00084619 0.00446367]
 [0.00929346 0.00667294 0.00887731 ... 0.00376071 0.00107242 0.00782623]]
starting seed: 2
[[-0.57312139 -0.97461586 -0.46

In [34]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [3 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,defense_time_threshold,defender_utility,support,runtime
0,1,8,3,1,-0.020930,2,2.130541
1,2,8,3,1,-0.021068,2,2.216249
2,3,8,3,1,-0.021014,1,2.185663
3,4,8,3,1,-0.021333,3,2.403166
4,5,8,3,1,-0.020914,1,1.997962
5,6,8,3,1,-0.020962,1,2.882317
6,7,8,3,1,-0.021171,1,2.685996
7,8,8,3,1,-0.021207,4,2.930520
8,9,8,3,1,-0.020962,1,2.315993
9,10,8,3,1,-0.020994,2,1.839676


In [35]:
df.to_csv("ISG_NFG_SSE_RANDOM_MATRIX.csv")

In [4]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_feature_value":  1, 
    "defender_feature_value": 100, 
    "defender_step_cost": 2,
    "alpha":.5
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

supports=[]
dts = []

dus = []
runtimes = []


for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
    isg.generate(num_attackers=1, 
                 num_defenders=3, 
                 home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=False,
                 randomize_target_utility_matrix=True,
                 generate_actions=False, 
                 force_return=True, 
                 defense_time_threshold=1, 
                 **schedule_form_kwargs,
                 **general_sum_kwargs, 
                )
    print(isg.schedule_form_dict["target_utilities"])
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(isg.schedule_form_dict["defender_utility_matrix"],isg.schedule_form_dict["attacker_utility_matrix"])
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
[[-1.22947276e+03 -6.10632851e+02 -2.08010566e+03 -1.46347808e+03
  -1.78090726e+03 -1.89193699e+03 -1.70030505e+03 -1.37527799e+03
  -1.27079887e+03 -9.80970216e+02 -1.22504010e+03 -6.82258938e+02
  -1.66318712e+03 -2.88681751e+02 -2.02445905e+03 -7.12358000e+02
  -1.22889575e+03 -9.40422366e+02 -1.79390208e+03 -1.67614483e+03
  -4.46548633e+02 -1.04757146e+02 -1.44084744e+03]
 [-6.67766175e+02 -2.92208037e+02 -2.55038138e+02 -1.90682000e+03
  -2.00065403e+03 -1.73382739e+03 -2.88630606e+02 -1.87967814e+03
  -1.22113671e+03 -1.25919635e+02 -9.92501088e+02 -6.68675150e+02
  -1.43658017e+03 -6.79644392e+02 -3.77419696e+02 -2.04302474e+03
  -5.49790305e+02 -6.27271550e+01 -5.53827444e+02 -1.50813821e+03
  -4.69941610e+02 -1.86972298e+03 -1.16648439e+03]
 [ 1.89384287e+01  6.39072406e+00  6.27159254e+00  3.05302370e+00
   7.95151600e-01  1.42505463e+01  4.71793104e+00  5.81805213e+00
   1.04297590e+01  1.48877683e+00  1.21139456e+01  3.39376038e+00
   1.24238309e+01  1.46


KeyboardInterrupt



In [38]:
len(defender_actions)

636056

In [9]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [3 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,defense_time_threshold,defender_utility,support,runtime
0,1,8,3,1,-0.434152,2,1.800039
1,2,8,3,1,-0.331237,3,1.841008
2,3,8,3,1,-0.451299,2,1.658311
3,4,8,3,1,-0.451827,4,1.702726
4,5,8,3,1,-0.454954,2,1.597217
5,6,8,3,1,-0.460837,3,1.837959
6,7,8,3,1,-0.455470,2,2.204338
7,8,8,3,1,-0.460089,2,1.811815
8,9,8,3,1,-0.448280,4,2.016663
9,10,8,3,1,-0.409055,2,1.643182


In [9]:
df.to_csv("ISG_NFG_SSE_RANDOM_TARGET_VALUES.csv")